# Cifar-10 이미지 분류
### 전체 데이터 사용
### Data Augmentation 적용
### 출처: [Achieving 90% accuracy in Object Recognition ](https://appliedmachinelearning.blog/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/)

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.regularizers import l2
import numpy as np

#### 자료형 변환 및 스케일링
- X: 실수형으로 정규화
- Y: 1-hot encoding
    * airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

In [2]:
(X_train, y_train0), (X_test, y_test0) = cifar10.load_data()
print(X_train.shape, X_train.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test.shape, X_test.dtype)
print(y_test0.shape, y_test0.dtype)

(50000, 32, 32, 3) uint8
(50000, 1) uint8
(10000, 32, 32, 3) uint8
(10000, 1) uint8


In [3]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

print(X_train.shape, X_train.dtype)

(50000, 32, 32, 3) float32


In [4]:
Y_train = tf.keras.utils.to_categorical(y_train0, 10)
Y_test = tf.keras.utils.to_categorical(y_test0, 10)
Y_train[:4]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

#### 모형 구현

In [5]:
np.random.seed(0)
weight_decay = 1e-4

In [6]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay), 
                 input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(lr=0.001, decay=weight_decay), 
              metrics=['accuracy'])

In [8]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    if epoch > 40:
        lrate = 0.0003
    return lrate

In [9]:
#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(X_train)

In [10]:
## to save checkpoint to use later
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

modelpath = "model/Cifar-10-best.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                    steps_per_epoch=X_train.shape[0] // 64, epochs=300,
                    verbose=1, validation_data=(X_test, Y_test),
                    callbacks=[LearningRateScheduler(lr_schedule),checkpointer,early_stopping_callback])

Epoch 1/300
780/781 [============================>.] - ETA: 0s - loss: 1.1630 - accuracy: 0.6190
Epoch 00001: val_loss improved from 1.23794 to 0.97928, saving model to model/Cifar-10-best.hdf5
781/781 [==============================] - 35s 45ms/step - loss: 1.1628 - accuracy: 0.6190 - val_loss: 0.9793 - val_accuracy: 0.6924 - lr: 0.0010
Epoch 2/300
780/781 [============================>.] - ETA: 0s - loss: 1.0198 - accuracy: 0.6694
Epoch 00002: val_loss improved from 0.97928 to 0.93154, saving model to model/Cifar-10-best.hdf5
781/781 [==============================] - 35s 44ms/step - loss: 1.0201 - accuracy: 0.6691 - val_loss: 0.9315 - val_accuracy: 0.7113 - lr: 0.0010
Epoch 3/300
780/781 [============================>.] - ETA: 0s - loss: 0.9357 - accuracy: 0.7022
Epoch 00003: val_loss did not improve from 0.93154
781/781 [==============================] - 35s 45ms/step - loss: 0.9356 - accuracy: 0.7023 - val_loss: 1.0672 - val_accuracy: 0.6918 - lr: 0.0010
Epoch 4/300
780/781 [=====

780/781 [============================>.] - ETA: 0s - loss: 0.5424 - accuracy: 0.8432
Epoch 00027: val_loss did not improve from 0.53670
781/781 [==============================] - 35s 45ms/step - loss: 0.5423 - accuracy: 0.8433 - val_loss: 0.5434 - val_accuracy: 0.8483 - lr: 0.0010
Epoch 28/300
780/781 [============================>.] - ETA: 0s - loss: 0.5343 - accuracy: 0.8477
Epoch 00028: val_loss improved from 0.53670 to 0.51214, saving model to model/Cifar-10-best.hdf5
781/781 [==============================] - 35s 44ms/step - loss: 0.5342 - accuracy: 0.8477 - val_loss: 0.5121 - val_accuracy: 0.8554 - lr: 0.0010
Epoch 29/300
780/781 [============================>.] - ETA: 0s - loss: 0.5264 - accuracy: 0.8465
Epoch 00029: val_loss did not improve from 0.51214
781/781 [==============================] - 35s 45ms/step - loss: 0.5264 - accuracy: 0.8465 - val_loss: 0.5669 - val_accuracy: 0.8407 - lr: 0.0010
Epoch 30/300
780/781 [============================>.] - ETA: 0s - loss: 0.5222 - a

Epoch 54/300
780/781 [============================>.] - ETA: 0s - loss: 0.4333 - accuracy: 0.8782
Epoch 00054: val_loss improved from 0.46226 to 0.45674, saving model to model/Cifar-10-best.hdf5
781/781 [==============================] - 35s 45ms/step - loss: 0.4334 - accuracy: 0.8781 - val_loss: 0.4567 - val_accuracy: 0.8768 - lr: 3.0000e-04
Epoch 55/300
781/781 [==============================] - ETA: 0s - loss: 0.4313 - accuracy: 0.8785
Epoch 00055: val_loss did not improve from 0.45674
781/781 [==============================] - 35s 44ms/step - loss: 0.4313 - accuracy: 0.8785 - val_loss: 0.4591 - val_accuracy: 0.8745 - lr: 3.0000e-04
Epoch 56/300
780/781 [============================>.] - ETA: 0s - loss: 0.4275 - accuracy: 0.8779
Epoch 00056: val_loss did not improve from 0.45674
781/781 [==============================] - 35s 45ms/step - loss: 0.4278 - accuracy: 0.8778 - val_loss: 0.4660 - val_accuracy: 0.8749 - lr: 3.0000e-04
Epoch 57/300
780/781 [============================>.] - E

Epoch 81/300
781/781 [==============================] - ETA: 0s - loss: 0.4046 - accuracy: 0.8852
Epoch 00081: val_loss did not improve from 0.44842
781/781 [==============================] - 35s 45ms/step - loss: 0.4046 - accuracy: 0.8852 - val_loss: 0.4513 - val_accuracy: 0.8764 - lr: 3.0000e-04
Epoch 82/300
780/781 [============================>.] - ETA: 0s - loss: 0.4077 - accuracy: 0.8844
Epoch 00082: val_loss did not improve from 0.44842
781/781 [==============================] - 35s 44ms/step - loss: 0.4076 - accuracy: 0.8844 - val_loss: 0.4515 - val_accuracy: 0.8777 - lr: 3.0000e-04
Epoch 83/300
780/781 [============================>.] - ETA: 0s - loss: 0.4080 - accuracy: 0.8838
Epoch 00083: val_loss improved from 0.44842 to 0.44707, saving model to model/Cifar-10-best.hdf5
781/781 [==============================] - 35s 45ms/step - loss: 0.4083 - accuracy: 0.8837 - val_loss: 0.4471 - val_accuracy: 0.8783 - lr: 3.0000e-04
Epoch 84/300
781/781 [==============================] - E

In [ ]:
from tensorflow.keras.models import load_model
del model
model = load_model('model/Cifar-10-best.hdf5')

#### 모델 평가하기

In [11]:
scores = model.evaluate(X_test, Y_test, batch_size=128, verbose=2)
print('\nAccuracy: %.4f' % scores[1])

10000/10000 - 3s - loss: 0.4770 - acc: 0.8714

Accuracy: 0.87140


#### 모델 저장하기

In [12]:
model.save_weights('model/cifar10-full-v3.h5') 